In [1]:
pip install kaggle


In [9]:
pip install scikit-learn


In [10]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d kazanova/sentiment140

 78% 63.0M/80.9M [00:00<00:00, 231MB/s]
100% 80.9M/80.9M [00:00<00:00, 223MB/s]


In [4]:
from zipfile import ZipFile
dataset ='/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is Extracted')

The dataset is Extracted


In [16]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO.8859.1')

In [20]:
twitter_data.shape

(1599999, 6)

In [22]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [23]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding = 'ISO.8859.1')

In [24]:
twitter_data.shape

(1600000, 6)

In [25]:
twitter_data.shape

(1600000, 6)

In [26]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [31]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [32]:
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [34]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [36]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [39]:
port_stem = PorterStemmer()

In [43]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]','',content)
  stemmed_content =stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ''.join(stemmed_content)

  return stemmed_content

In [44]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [46]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [47]:
print(twitter_data['stemmed_content'])



0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [48]:
print(twitter_data['target'])



0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [49]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [50]:
print(X)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [51]:
print(Y)

[0 0 0 ... 1 1 1]


In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [57]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [58]:
print(X_test)

['mmangenmdoingfineihaventhadmuchtimetochatontwitterhubbyisbackforthesummeramptendstodominatemyfreetim'
 'atahsmayshowwruthkimampgeoffreysanhueza'
 'ishataramaybeitwasonlyabayareathangdammit' ...
 'destinineverthelesshoorayformembersandhaveawonderfulandsafetrip'
 'notfeelingtoowel' 'supersandrothanky']


In [59]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [60]:
print(X_train)

  (0, 4269)	1.0
  (1, 370478)	1.0
  (2, 259878)	1.0
  (3, 544662)	1.0
  (4, 1125692)	1.0
  (5, 183084)	1.0
  (6, 655943)	1.0
  (7, 427969)	1.0
  (8, 372293)	1.0
  (9, 414371)	1.0
  (10, 453206)	1.0
  (11, 946311)	1.0
  (12, 1165280)	1.0
  (13, 334893)	1.0
  (14, 1029544)	1.0
  (15, 1251389)	1.0
  (16, 26694)	1.0
  (17, 620642)	1.0
  (18, 371055)	1.0
  (19, 166384)	1.0
  (20, 939661)	1.0
  (21, 392755)	1.0
  (22, 288391)	1.0
  (23, 202689)	1.0
  (24, 1253472)	1.0
  :	:
  (1279975, 97244)	1.0
  (1279976, 1110776)	1.0
  (1279977, 938969)	1.0
  (1279978, 783480)	1.0
  (1279979, 800934)	1.0
  (1279980, 865418)	1.0
  (1279981, 587370)	1.0
  (1279982, 952975)	1.0
  (1279983, 412833)	1.0
  (1279984, 81684)	1.0
  (1279985, 59066)	1.0
  (1279986, 1015971)	1.0
  (1279987, 777243)	1.0
  (1279988, 1218709)	1.0
  (1279989, 450972)	1.0
  (1279990, 465151)	1.0
  (1279991, 249222)	1.0
  (1279992, 1169847)	1.0
  (1279993, 1170137)	1.0
  (1279994, 75555)	1.0
  (1279995, 1106989)	1.0
  (1279996, 561111)	1

In [61]:
print(X_test)

  (22, 137709)	1.0
  (36, 790487)	1.0
  (55, 1232068)	1.0
  (104, 806213)	1.0
  (316, 675309)	1.0
  (317, 183110)	1.0
  (326, 530902)	1.0
  (343, 483231)	1.0
  (356, 1004786)	1.0
  (405, 294045)	1.0
  (411, 754846)	1.0
  (412, 671785)	1.0
  (503, 922733)	1.0
  (575, 484365)	1.0
  (576, 68315)	1.0
  (585, 553737)	1.0
  (597, 437993)	1.0
  (704, 437477)	1.0
  (730, 522522)	1.0
  (735, 1190472)	1.0
  (763, 201539)	1.0
  (851, 230654)	1.0
  (887, 945330)	1.0
  (905, 1080766)	1.0
  (965, 64114)	1.0
  :	:
  (319388, 469685)	1.0
  (319391, 363430)	1.0
  (319401, 69390)	1.0
  (319403, 1010168)	1.0
  (319419, 435823)	1.0
  (319442, 1154002)	1.0
  (319462, 331998)	1.0
  (319474, 363236)	1.0
  (319550, 1020181)	1.0
  (319573, 1087679)	1.0
  (319595, 377142)	1.0
  (319630, 416243)	1.0
  (319632, 324586)	1.0
  (319640, 333810)	1.0
  (319649, 564791)	1.0
  (319673, 473916)	1.0
  (319679, 363450)	1.0
  (319687, 874322)	1.0
  (319700, 937693)	1.0
  (319712, 230590)	1.0
  (319779, 553484)	1.0
  (319838

Logistic Regression #machine Model

In [62]:
model = LogisticRegression(max_iter=1000)

In [63]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [64]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [65]:
print('Accuracy score on the traing data:',training_data_accuracy)

Accuracy score on the traing data: 0.99812265625


In [69]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [70]:
print('Accuracy score on the traing data:',test_data_accuracy)

Accuracy score on the traing data: 0.512653125


Model accuracy = 51.26%

saving the trained predictions

In [71]:
import pickle

In [72]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

usnig the saved model for future predictions

In [73]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [76]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('positive Tweet')

1
[1]
positive Tweet


In [77]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('positive Tweet')

0
[1]
positive Tweet
